In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from scipy.io import mmread
import scipy
from pprint import pprint
from sklearn.preprocessing import normalize
from ConfigParser import SafeConfigParser
import numpy as np
import time

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from eval_util import *

import cvxopt
import numpy as np
import pandas as pd
import random
from cvxopt import matrix, solvers,spmatrix
import scipy.sparse as sparse
import time

#Configuration

In [70]:
config = '/home/ellery/wikimedia/missing_articles/missing_articles.ini'
cp = SafeConfigParser()
cp.read(config)

embedding_dir = 'en_lda'
embedding_path = os.path.join(cp.get('general', 'local_data_dir'), embedding_dir)



#Load Editor Contribution Data

In [104]:
contributon_dir = 'en-es'
contributon_path = os.path.join(cp.get('general', 'local_data_dir'), contributon_dir)
contribution_file = os.path.join(contributon_path, cp.get('eval', 'train'))  

def contribution_iter(contribution_file, kmax=10):
    with open(contribution_file) as f:
        for line in f:
            try:
                contributions = json.loads(line)
            except:
                print "decode error"
                continue
            contributions['contributions'] = contributions['contributions'][-kmax:]
            yield contributions         


it = contribution_iter(contribution_file)
 

#Load Doc Topic Matrices

In [72]:
# later translation_dict_file will point to a reduced version of ranked_missing_items
translation_dict_file = os.path.join(contributon_path, cp.get('missing', 'ranked_missing_items'))
id2sname, id2importance = get_translation_maps(translation_dict_file)

embedding_dict_file = os.path.join(embedding_path, cp.get('LDA', 'article2index'))
id2index, index2id, id2uname = get_universal_maps(embedding_dict_file)

mm_file = os.path.join(embedding_path, cp.get('LDA', 'doc2topic'))
f = open(mm_file)
M = mmread(f).tocsr()
M = normalize(M, norm='l2', axis=1)

KeyboardInterrupt: 

#Augment Contributions with missing article affinities

In [105]:
n = 200
MI = M
MR = reduce_and_reweight(M, id2importance, id2index, index2id)
num_to_rank = 100
min_score = 0.6
contributions = {}

for i in range(n):
    contribution = next(it)
    contribution['affinities'] = recommend(MI, MR, id2index, index2id, None, contribution, get_weighted_average_interest_vector, num_to_rank, min_score)
    contributions[contribution['uname']] = contribution

# Set up data structures for matching

In [106]:
editor_ids = contributions.keys()
article_ids = []
affinities = {}
for uname, c in contributions.iteritems():
    for t in c['affinities']:
        article_ids.append(t[0])
        edge = (uname, t[0])
        affinities[edge] = t[1]
article_ids = set(article_ids)
print ('Numer of unique articles: %d' % len(article_ids))

Numer of unique articles: 16797


#Matching Code

In [5]:
# set nore map and constraints 
from scipy.sparse import csr_matrix
def match(editor_ids, article_ids, affinities, k ):
    node2idx = {}
    idx2node = []
    b = []
    i = 0
    for ed in editor_ids:
        node2idx[ed] = i
        idx2node.append(ed)
        b.append(k)
        i+=1

    for a in article_ids:
        node2idx[a] = i
        idx2node.append(a)
        b.append(0.0)
        i+=1

    edge2idx = {}
    idx2edge = []
    c = []


    j = 0
    for e in affinities.keys():
        edge2idx = j
        idx2edge.append(e)
        j+=1
        c.append(affinities[e])

    for e in idx2edge:
        b.append(1.0)

    for e in idx2edge:
        b.append(0.0)


    rows = []
    cols = []
    values = []

    for i, n in enumerate(idx2node):
        for j, e in enumerate(idx2edge):
            if n==e[0]:
                rows.append(i)
                cols.append(j)
                values.append(1.0)
            elif n==e[1]:
                rows.append(i)
                cols.append(j)
                values.append(-1.0)

    num_nodes = len(idx2node)
    num_edges = len(idx2edge)

    for i in range(num_nodes, num_nodes + num_edges):
        rows.append(i)
        cols.append(i-num_nodes)
        values.append(1.0)

    for i in range(num_nodes + num_edges, num_nodes + 2*num_edges):
        rows.append(i)
        cols.append(i-num_nodes - num_edges)
        values.append(-1.0)
    
    As = csr_matrix((vlaues, (rows, cols)))
    print (As)
    A =  spmatrix(values, rows, cols)

    t1 = time.time()
    sol=solvers.lp(matrix(np.array(c)),A,matrix(b))
    t2 = time.time()
    print ('Optimization took %d seconds' % (t2-t1))
    non_zero, _ = np.where(np.round(sol['x']) ==1.0)
    print ("Objective %f " % np.array(c).dot(sol['x']))
    d = pd.DataFrame(idx2edge)
    return d.ix[non_zero]


# Inspect Matching

In [6]:
d = match(editor_ids, article_ids, affinities, 5 )

NameError: name 'editor_ids' is not defined

In [108]:
for i, g in enumerate(d.groupby(0)):
    dm = g[1]
    dm.columns = ['uname', 'id']
    dm['name'] = dm['id'].apply(lambda x: id2sname[x])
    uname = dm['uname'][dm.index[0]]
    print "Editor %s\n" % uname
    print "Past Contributions:"
    for c in contributions[uname]['contributions']:
        print c['page_title']
    print "\nRecommended Articles:"  
    for a in list(dm['name']):
        print a
    if i ==50:
        break
    print "\n\n"

Editor 000peter

Past Contributions:
Reino_de_Dinamarca
John_Wesley
Margarita_II_de_Dinamarca
Der_er_et_Yndigt_Land
Dinamarca

Recommended Articles:
A Story about a Darning-needle
Order of Woodcraft Chivalry



Editor Alan.Crazitho

Past Contributions:
4_de_octubre
Día_de_la_Raza

Recommended Articles:
Eric Rosenthal
Julian Rios
Berrigan Brothers
Sheyla
Bernardo de Gálvez (statue)



Editor Alegarsan

Past Contributions:
Emilio_Arrieta
Cantar_de_mio_Cid
Felipe_Pedrell
Juan_de_Anchieta
Ricardo_García_Damborenea
Federico_Olmeda
Tourdion
Casina
A_la_nanita_nana
Maite

Recommended Articles:
Bartomeu Càrceres
Juan Arañés
Endecha
Cor de la Generalitat Valenciana
Coplas de Purim



Editor Alejandronoize

Past Contributions:
Steve_Angello
Wolfgang_Gartner
Daft_Punk
Martin_Solveig
Calvin_Harris
Knife_Party
Chuckie
Porter_Robinson
Axwell
Dada_Life

Recommended Articles:
Whiteboy (song)
Jody Wisternoff
Breaking Benjamin discography
The One Man Band Broke Up
Playa (band)



Editor Alexthenh

Past 

/home/ellery/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
